In [ ]:
# Copyright 2026 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI Model Garden - Hugging Face Pytorch Inference Deployment

<table><tbody><tr>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fcommunity%2Fmodel_garden%2Fmodel_garden_huggingface_pytorch_inference_deployment.ipynb">
      <img alt="Google Cloud Colab Enterprise logo" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" width="32px"><br> Run in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_huggingface_pytorch_inference_deployment.ipynb">
      <img alt="GitHub logo" src="https://github.githubassets.com/assets/GitHub-Mark-ea2971cee799.png" width="32px"><br> View on GitHub
    </a>
  </td>
</tr></tbody></table>

## Overview

This notebook demonstrates deploying [distilbert/distilbert-base-uncased-finetuned-sst-2-english](https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english) with [Hugging Face Pytorch Inference](https://github.com/huggingface/Google-Cloud-Containers/tree/main/containers/pytorch/inference). In additional to `distilbert/distilbert-base-uncased-finetuned-sst-2-english`, You can view and change the code to deploy a different Hugging Face Pytorch model with appropriate machine specs. **Note that not all Pytorch models are supported by this serving container.**


### Objective

- Download and deploy the `distilbert/distilbert-base-uncased-finetuned-sst-2-english` model with Hugging Face Pytorch Inference
- Send prediction request to the deployed endpoint

### File a bug

File a bug on [GitHub](https://github.com/GoogleCloudPlatform/vertex-ai-samples/issues/new) if you encounter any issue with the notebook.

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing) and use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.

## Run the notebook

In [ ]:
# @title Setup Google Cloud project

# @markdown 1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

# @markdown 2. **[Optional]** Set region. If not set, the region will be set automatically according to Colab Enterprise environment.

REGION = ""  # @param {type:"string"}

# @markdown 3. If you want to run predictions with A100 80GB or H100 GPUs, we recommend using the regions listed below. **NOTE:** Make sure you have associated quota in selected regions. Click the links to see your current quota for each GPU type: [Nvidia A100 80GB](https://console.cloud.google.com/iam-admin/quotas?metric=aiplatform.googleapis.com%2Fcustom_model_serving_nvidia_a100_80gb_gpus), [Nvidia H100 80GB](https://console.cloud.google.com/iam-admin/quotas?metric=aiplatform.googleapis.com%2Fcustom_model_serving_nvidia_h100_gpus). You can request for quota following the instructions at ["Request a higher quota"](https://cloud.google.com/docs/quota/view-manage#requesting_higher_quota).

# @markdown | Machine Type | Accelerator Type | Recommended Regions |
# @markdown | ----------- | ----------- | ----------- |
# @markdown | a2-ultragpu-1g | 1 NVIDIA_A100_80GB | us-central1, us-east4, europe-west4, asia-southeast1, us-east4 |
# @markdown | a3-highgpu-2g | 2 NVIDIA_H100_80GB | us-west1, asia-southeast1, europe-west4 |
# @markdown | a3-highgpu-4g | 4 NVIDIA_H100_80GB | us-west1, asia-southeast1, europe-west4 |
# @markdown | a3-highgpu-8g | 8 NVIDIA_H100_80GB | us-central1, europe-west4, us-west1, asia-southeast1 |
# Upgrade Vertex AI SDK.
! pip3 install --upgrade --quiet 'google-cloud-aiplatform>=1.64.0'
! git clone https://github.com/GoogleCloudPlatform/vertex-ai-samples.git

import importlib
import os
from typing import Tuple

from google.cloud import aiplatform

common_util = importlib.import_module(
    "vertex-ai-samples.notebooks.community.model_garden.docker_source_codes.notebook_util.common_util"
)


# Get the default cloud project id.
PROJECT_ID = os.environ["GOOGLE_CLOUD_PROJECT"]

# Get the default region for launching jobs.
if not REGION:
    REGION = os.environ["GOOGLE_CLOUD_REGION"]

# Initialize Vertex AI API.
print("Initializing Vertex AI API.")
aiplatform.init(project=PROJECT_ID, location=REGION)

! gcloud config set project $PROJECT_ID

HF_TOKEN = ""

# @markdown Set `use_dedicated_endpoint` to False if you don't want to use [dedicated endpoint](https://cloud.google.com/vertex-ai/docs/general/deployment#create-dedicated-endpoint).
use_dedicated_endpoint = True  # @param {type:"boolean"}

SERVICE_ACCOUNT = ""

In [ ]:
# @title Deploy with Hugging Face Pytorch Inference

# @markdown This section downloads the `distilbert/distilbert-base-uncased-finetuned-sst-2-english` model from Hugging Face and deploys it to a Vertex AI Endpoint.
# @markdown It takes ~20 minutes to complete the deployment.

MODEL_ID = "distilbert/distilbert-base-uncased-finetuned-sst-2-english"  # @param {type: "string", isTemplate: true}
TASK = "text-classification"  # @param {type: "string", isTemplate: true}

# The pre-built serving docker images for Hugging Face Pytorch Inference.
SERVE_DOCKER_URI = "us-docker.pkg.dev/deeplearning-platform-release/vertex-model-garden/hf-inference-toolkit.cu125.0-1.ubuntu2204.py311:model-garden.hf-inference-toolkit-0-1-release_20251206.00_p0"

machine_type = "g2-standard-8"  # @param {type: "string", isTemplate: true}
accelerator_type = "NVIDIA_L4"  # @param ["NVIDIA_L4", "None"] {isTemplate: true}
accelerator_count = 1  # @param {type: "integer", isTemplate: true}

if accelerator_type == "None":
    accelerator_type = ""
    accelerator_count = 0

if accelerator_type:
    common_util.check_quota(
        project_id=PROJECT_ID,
        region=REGION,
        accelerator_type=accelerator_type,
        accelerator_count=accelerator_count,
        is_for_training=False,
    )

models, endpoints = {}, {}

GCS_PREFIX = "gs://"


def is_gcs_path(path: str) -> bool:
    return path.startswith(GCS_PREFIX)


def deploy_model(
    model_name: str,
    model_id: str,
    publisher: str,
    publisher_model_id: str,
    task: str,
    machine_type: str = "g2-standard-12",
    accelerator_type: str = "NVIDIA_L4",
    accelerator_count: int = 1,
    use_dedicated_endpoint: bool = False,
) -> Tuple[aiplatform.Model, aiplatform.Endpoint]:
    """Create a Vertex AI Endpoint and deploy the specified model to the endpoint."""

    endpoint = aiplatform.Endpoint.create(
        display_name=f"{model_name}-endpoint",
        dedicated_endpoint_enabled=use_dedicated_endpoint,
    )
    serving_env = {
        "HF_TASK": task,
        "MODEL_ID": model_id,
        "DEPLOY_SOURCE": "notebook",
    }
    if not is_gcs_path(model_id):
        serving_env.update(
            {
                "HF_MODEL_ID": model_id,
            }
        )
        try:
            if HF_TOKEN:
                serving_env.update(
                    {
                        "HF_TOKEN": HF_TOKEN,
                    }
                )
        except:
            pass

    model = aiplatform.Model.upload(
        display_name=model_name,
        artifact_uri=model_id if is_gcs_path(model_id) else None,
        serving_container_image_uri=SERVE_DOCKER_URI,
        serving_container_ports=[7080],
        serving_container_predict_route="/pred",
        serving_container_health_route="/ping",
        serving_container_environment_variables=serving_env,
        model_garden_source_model_name=(
            f"publishers/{publisher}/models/{publisher_model_id}"
        ),
    )

    model.deploy(
        endpoint=endpoint,
        machine_type=machine_type,
        accelerator_type=accelerator_type,
        accelerator_count=accelerator_count,
        deploy_request_timeout=1800,
        service_account=SERVICE_ACCOUNT,
        system_labels={
            "NOTEBOOK_NAME": "model_garden_huggingface_pytorch_inference_deployment.ipynb",
        },
    )
    return model, endpoint


models["pytorch_inference_gpu"], endpoints["pytorch_inference_gpu"] = deploy_model(
    model_name=common_util.get_job_name_with_datetime(prefix=MODEL_ID),
    model_id=MODEL_ID,
    publisher="hf-distilbert",
    publisher_model_id="distilbert-base-uncased-finetuned-sst-2-english",
    task=TASK,
    machine_type=machine_type,
    accelerator_type=accelerator_type,
    accelerator_count=accelerator_count,
    use_dedicated_endpoint=use_dedicated_endpoint,
)

# @markdown Click "Show Code" to see more details.

In [ ]:
# @title Predict

# @markdown Once deployment succeeds, you can send requests to the endpoint with input text.

# @markdown This example uses the following input:

# @markdown > \["I love this product", "I hate this product"\]

# Loads an existing endpoint instance using the endpoint name:
# - Using `endpoint_name = endpoints["pytorch_inference_gpu"].name` allows us to get the endpoint name of
#   the endpoint `endpoint` created in the cell above.
# - Alternatively, you can set `endpoint_name = "1234567890123456789"` to load
#   an existing endpoint with the ID 1234567890123456789.

# You may uncomment the code below to load an existing endpoint.
# endpoint_name = ""  # @param {type:"string"}
# aip_endpoint_name = (
#     f"projects/{PROJECT_ID}/locations/{REGION}/endpoints/{endpoint_name}"
# )
# endpoints["pytorch_inference_gpu"] = aiplatform.Endpoint(aip_endpoint_name)
# print("Using this existing endpoint from a different session: {aip_endpoint_name}")

instances = ["I love this product", "I hate this product"]

response = endpoints["pytorch_inference_gpu"].predict(
    instances=instances, use_dedicated_endpoint=use_dedicated_endpoint
)
print(response.predictions)

## Clean up resources

In [ ]:
# @title Delete the models and endpoints
# @markdown  Delete the experiment models and endpoints to recycle the resources
# @markdown  and avoid unnecessary continuous charges that may incur.

# Undeploy model and delete endpoint.
for endpoint in endpoints.values():
    endpoint.delete(force=True)

# Delete models.
for model in models.values():
    model.delete()